In [1]:
### Enter topic of interest

topic = 'AI Research by Google'
topic = 'AI-powered Self-Service Analytics'

topic = 'precision medicine and its impact on healthcare'

In [2]:
from datetime import datetime
from helper import *

In [3]:
output_file = f"blog__{normalize_text(topic)}.md"
output_file

'blog__precision-medicine-and-its-impact-on-healthcare.md'

## Use CrewAI

- [GitHub](https://github.com/joaomdmoura/crewai/)
- [Docs](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/)

### Quick Overview

Top-down approach
- Goal: What is the crew for? (define the use-case)
- Define all the Tasks required to achieve the goal? (including human review/action)
- Define and Assign the agent(s) required to complete each task
- Define Tools (maybe used by Agent/Task)
- Kickoff Crew process

In [4]:
from api_key_store import ApiKeyStore
s = ApiKeyStore()

openai_api_key = s.get_api_key(provider="OPENAI")
serp_api_key = s.get_api_key(provider="SerpApi")

In [5]:
s.api_providers

dict_keys(['OPENAI', 'MICROSOFT', 'GOOGLE', 'HUGGING_FACE', 'WOLFRAM_ALPHA', 'ANTHROPIC', 'COHERE', 'SerpApi', 'Browserless', 'DeepL'])

In [6]:
import os
os.environ["SERPER_API_KEY"] = serp_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [7]:
from crewai import Agent
from crewai_tools import SerperDevTool

### Step 1: Assemble Your Agents

#### Define Tools

In [8]:
search_tool = SerperDevTool()

#### Define Agents

In [9]:
# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

In [10]:
# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

### Step 2: Define the Tasks

In [11]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file=output_file  # Example of output customization
)

### Step 3: Form the Crew

In [12]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True, # False,  # 
  cache=True, # False,   # 
  max_rpm=50,
  # share_crew=True
)

### Step 4: Kick It Off

In [13]:
from time import time
ts_start = time()

In [14]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': topic})

ts_stop = time()
print(result)



> Entering new CrewAgentExecutor chain...
The provided context suggests that Generative AI, including its subset Generative BI, could be a significant trend in precision medicine. However, I need to validate this hypothesis, understand its advantages, risks and market opportunities. I should also consider any potential privacy and security concerns related to the usage of Generative AI in precision medicine. I will begin by searching the internet to gather information about the current trends in precision medicine, particularly focusing on Generative AI.

Action: Search the internet
Action Input: {"search_query": "Generative AI in precision medicine trends, advantages, risks, and market opportunities"} 


Search results: Title: Generative AI in health care: Opportunities, challenges, and policy
Link: https://www.brookings.edu/articles/generative-ai-in-health-care-opportunities-challenges-and-policy/
Snippet: Generative AI poses many benefits for the healthcare industry, but transpare

In [15]:
elapsed_time = ts_stop - ts_start

In [16]:
print(f"""Done researching topic: "{topic}" in {elapsed_time:.2f} sec,
the writeup is in file: '{output_file}'
""")

Done researching topic: "precision medicine and its impact on healthcare" in 147.55 sec,
the writeup is in file: 'blog__precision-medicine-and-its-impact-on-healthcare.md'



In [17]:
!pwd

/home/papagame/projects/wgong/py4kids/lesson-18-ai/crewai/papagame
